In [ ]:
import numpy as np
import scipy.signal
import hfda
from petastorm import make_reader
import matplotlib.pyplot as plt
from scipy import interpolate
import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))
from shared_utils import TSD_cal as TSD

In [ ]:
sig_test = np.random.normal(0,1,5000)
plt.figure()
plt.plot(sig_test)

h = [0.001]
hprime = [0.005]
collect_I1 = {}
collect_I2 = {}
name_h = np.array([])
for i,j in zip(h,hprime):
 I1_c,I2_c,c = TSD.discrepancies_mean_curve(sig_test,500,i,j,1/500)
 #print(f"I1 for h = {i} and hprime = {j}: ",I1_c)
 #print(f"I2 for h = {i} and hprime = {j} : ",I2_c)
 collect_I1[f"h = {i} and hprime = {j}"] = I1_c
 collect_I2[f"h = {i} and hprime = {j}"] = I2_c
 name_h = np.append(name_h,f"h = {i} and hprime = {j}")

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
for nh in name_h:
    ax[0].plot(c,collect_I1[nh],label = nh )
    ax[1].plot(c,collect_I2[nh],label = nh )
ax[0].set_xlabel("segment length")
ax[0].set_ylabel("I1")
ax[0].grid()
ax[1].legend(loc = "best", bbox_to_anchor=(1,1))
ax[1].set_xlabel("segment length")
ax[1].set_ylabel("I2")
ax[1].grid()

###fix h and mix hprime
# h = [0.2,0.2,0.2,0.2,0.2,0.2,0.2]
# hprime = [0.2,0.1,0.05,0.04,0.03,0.02,0.01]
# collect_I1 = {}
# collect_I2 = {}
# name_h = np.array([])
# for i,j in zip(h,hprime):
#  I1_c,I2_c,c = discrepancies_mean_curve(sig_test,500,i,j)
#  #print(f"I1 for h = {i} and hprime = {j}: ",I1_c)
#  #print(f"I2 for h = {i} and hprime = {j} : ",I2_c)
#  collect_I1[f"h = {i} and hprime = {j}"] = I1_c
#  collect_I2[f"h = {i} and hprime = {j}"] = I2_c
#  name_h = np.append(name_h,f"h = {i} and hprime = {j}")

# fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
# for nh in name_h:
#     ax[0].plot(c,collect_I1[nh],label = nh )
#     ax[1].plot(c,collect_I2[nh],label = nh )
# ax[0].set_xlabel("segment length")
# ax[0].set_ylabel("I1")
# ax[0].grid()
# ax[1].legend(loc = "best", bbox_to_anchor=(1,1))
# ax[1].set_xlabel("segment length")
# ax[1].set_ylabel("I2")
# ax[1].grid()


# ##fixh prime and mix h

# hprime = [0.01,0.01,0.01,0.01,0.01,0.01,0.01]
# h = [0.2,0.1,0.05,0.04,0.03,0.02,0.01]
# collect_I1 = {}
# collect_I2 = {}
# name_h = np.array([])
# for i,j in zip(h,hprime):
#  I1_c,I2_c,c = discrepancies_mean_curve(sig_test,500,i,j)
#  #print(f"I1 for h = {i} and hprime = {j}: ",I1_c)
#  #print(f"I2 for h = {i} and hprime = {j} : ",I2_c)
#  collect_I1[f"h = {i} and hprime = {j}"] = I1_c
#  collect_I2[f"h = {i} and hprime = {j}"] = I2_c
#  name_h = np.append(name_h,f"h = {i} and hprime = {j}")

# fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
# for nh in name_h:
#     ax[0].plot(c,collect_I1[nh],label = nh )
#     ax[1].plot(c,collect_I2[nh],label = nh )
# ax[0].set_xlabel("segment length")
# ax[0].set_ylabel("I1")
# ax[0].grid()
# ax[1].legend(loc = "best", bbox_to_anchor=(1,1))
# ax[1].set_xlabel("segment length")
# ax[1].set_ylabel("I2")
# ax[1].grid()

In [ ]:
###Test : Get data
path_formatted_glasgow = "/workspaces/maitrise/data/20220902_data_physio_formatted_merged/merged/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        break

print(data)
fs = data.sampling_frequency



In [ ]:
##Plot data :
def plot_ECG_signal(signal,name,length= data.signal_length,fs = data.sampling_frequency):
     x = np.array(range(0,length))
     x = x/fs     
     fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
     ax[0].plot(x,signal)
     ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
     ax[0].grid()
     ax[1].plot(x,signal)
     ax[1].set_title(f"Close up signal of Lead {name.decode('utf8')}")
     ax[1].grid()
     if len(x) == data.signal_length:
          ax[1].set_xlim([0,3])
     else :
          ax[1].set_xlim([0,x[-1]])
     plt.show()

ECG_signal = data.signal
ECG_lead = data.signal_names
dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]
     print(dico_ECG[i].shape)
     plot_ECG_signal(dico_ECG[i],i)

In [ ]:
##Plot TSD of ECG

TSD.TSD_plot(dico_ECG,ECG_lead,fs)
tab,val=TSD.TSD_index(dico_ECG,ECG_lead,fs)
for i in ECG_lead:
    print(f"Lead {i.decode('utf8')} TSD : ",tab[i][0])
print(val)


In [ ]:
##Test function : 

h = 0.001
hprime = 0.001
fs = data.sampling_frequency

for i in ECG_lead:
    I1_c,I2_c,c = TSD.discrepancies_mean_curve(dico_ECG[i],fs,h,hprime,1/fs)
    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
    ax[0].set_title(f"Lead {i.decode('utf8')}")
    ax[0].plot(c,I1_c)
    ax[0].set_xlabel("signal legnth [s]")
    ax[0].set_ylabel("I1 value")
    ax[0].grid()
    ax[1].set_title(f"Lead {i.decode('utf8')}")
    ax[1].plot(c,I2_c)
    ax[1].set_xlabel("signal legnth [s]")
    ax[1].set_ylabel("I2 value")
    ax[1].grid()
    


In [ ]:
##RMS:

fs = data.sampling_frequency
c_axis = np.linspace(1,int(data.signal_length/fs),100)

# def RMS_array_creator(dico_signal,name,c_ax,fs = data.sampling_frequency):
#     RMS_val = np.array([])
#     for j in c_ax:
#         #print("le temps : ",j)
#         D = TSD.TSD_ECG(dico_signal,name,j,fs)
#         RMS_val = np.append(RMS_val,TSD.RMS(D))

#     return RMS_val
 
plt.plot(c_axis,TSD.RMS_array_creator(dico_ECG,ECG_lead,c_axis,fs))
plt.title("Evolution of RMS value function of length of time series")
plt.xlabel("segment length")
plt.ylabel("RMS val")
plt.grid()
plt.show()

index_min = np.argmin(TSD.RMS_array_creator(dico_ECG,ECG_lead,c_axis,fs))
print("Behold! c* accoridng to RMS :",c_axis[index_min])

In [ ]:
##Graph comme proposer dans etude :

eps1 = np.linspace(0,1,100)
eps2 = np.linspace(0,0.01,100)
h = 0.001
hprime = 0.005
##TSD ECG article:

#c_axis = np.linspace(1,int(data.signal_length/fs),1000)
c_matrix = {}

x,y = np.meshgrid(eps1,eps2)

for i in ECG_lead:
    sig = dico_ECG[i]
    I1_c,I2_c,c = TSD.discrepancies_mean_curve(sig,fs,h,hprime,1/fs)
    I1c_r = I1_c[np.logical_and(I1_c>0,I1_c<=1)]
    I2c_r = I1_c[np.logical_and(I2_c>0,I2_c<=1)]
    c1 = c[np.logical_and(I1_c>0,I1_c<=1)]
    c2 = c[np.logical_and(I2_c>0,I2_c<=1)]
    c1x,c2y = np.meshgrid(c1,c2)
    z = np.empty((len(c1),len(c2)))
    for w in range(len(c1)):
        for j in range(len(c2)):
            z[w][j] = np.minimum(c1x[w][j],c2y[w][j])
    c_matrix[i] = z

##Now let's create da matrix:

RMS_matrix = np.empty((c_matrix[ECG_lead[0]].shape[0],c_matrix[ECG_lead[0]].shape[1]))
for i in range(c_matrix[ECG_lead[0]].shape[0]):
    for j in range(c_matrix[ECG_lead[0]].shape[1]):
        D = np.array([])
        for n in ECG_lead:
            c_lead = c_matrix[n][i][j] 
            w=1
            Ds = np.array([])
            sig = dico_ECG[n]
            while (w*np.abs(c_lead)*fs)<=len(sig):
                sig_c  = sig[int((w-1)*np.abs(c_lead)*fs):int((w)*np.abs(c_lead)*fs)]
                L1 = TSD.Lq_k(sig_c,1,fs)
                L2 = TSD.Lq_k(sig_c,2,fs)
                Dv = (np.log(L1)-np.log(L2))/(np.log(2))   
                Ds = np.append(Ds,Dv)
                w+=1
            D = np.append(D,np.mean(Ds))
        RMS_matrix[i,j] = TSD.RMS(D)

plt.imshow(RMS_matrix,interpolation="none",cmap="hot",extent = [0,1,0,1],aspect = 2)
pc = plt.pcolormesh(RMS_matrix,cmap = "hot")
plt.colorbar(pc)
plt.show()

In [ ]:
TSD.TSD_plot(dico_ECG,ECG_lead,1,data.sampling_frequency)

In [ ]:
###Function for TF phase shuffling (algo 1 of the article : "Testing for nonlinearity in time series:
#the method of surrogate data")
from random import uniform
from scipy.fftpack import rfft,irfft,fft, ifft

def Random_phase_shuffling(signal):
    fft_signal = rfft(signal)
    phase_fs = np.arctan2(fft_signal[2::2],fft_signal[1:-1:2])
    mag = np.sqrt((fft_signal[1:-1:2])**2+(fft_signal[2::2])**2)
    ##phase shuffler:
    #rng_phase = phase_fs.copy()
    #np.random.shuffle(rng_phase)
    rng_phase = np.random.uniform(-np.pi,np.pi,len(phase_fs))
    ##new signal : 
    fsrp = mag[:, np.newaxis] * np.c_[np.cos(phase_fs+rng_phase), np.sin(phase_fs+rng_phase)]
    #fsrp = mag[:, np.newaxis] * np.c_[np.cos(rng_phase), np.sin(rng_phase)]
    fsrp = np.r_[fft_signal[0], fsrp.ravel(), fft_signal[-1]]
    tsr = irfft(fsrp)
    return tsr

sig = dico_ECG[ECG_lead[6]]
new_sig = Random_phase_shuffling(sig)

fig,ax = plt.subplots(1,2,figsize = (20,10))
ax[0].plot(sig,alpha =0.6)
ax[0].set_title("Old lead")
ax[1].plot(new_sig, alpha =0.6)
ax[1].set_title("noisy lead")



In [ ]:
###Trial noise : 
D_lead = {}

Noisy_ECG = {}
for i in ECG_lead:
    Noisy_ECG[i] = Random_phase_shuffling(dico_ECG[i])

TSD.TSD_plot(Noisy_ECG,ECG_lead,1,data.sampling_frequency)

In [ ]:
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [ ]:
##SNR of each ECG signal : 

for i in ECG_lead:
    print(f"For lead {i.decode('utf8')} : SNR  =",signaltonoise(dico_ECG[i]))